# MNIST master

For a demo we shall solve the same digit recognition problem, but at a different scale
* images are now 28x28
* 10 different digits
* 50k samples

Before doing this homework, read some code examples written in tensorflow. There is a good repository with code examples: https://github.com/aymericdamien/TensorFlow-Examples. As we already know, we need many samples to learn :)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "xtick.alignment" on line 250 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

Bad key "ytick.alignment" on line 268 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import tensorflow as tf

In [3]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print(X_train.shape,y_train.shape)

(50000, 1, 28, 28) (50000,)


In [4]:
X_train = X_train.reshape(X_train.shape[0],784)
X_val = X_val.reshape(X_val.shape[0],784)
X_test = X_test.reshape(X_test.shape[0],784)

In [5]:
#defining placeholders for input and target
input_X = tf.placeholder(tf.float32, shape=[None, 784], 
                         name="X")
target_y = tf.placeholder(tf.int32, shape=[None], 
                          name="target_Y_integer")


Defining network architecture

In [6]:
x =tf.reshape(input_X, shape=[-1, 28, 28, 1],name='reshape_X')

conv1 = tf.layers.conv2d(x, 32, 5, activation=None)
batch1 = tf.layers.batch_normalization(conv1)
act1 = tf.nn.relu(batch1) 
conv1 = tf.layers.max_pooling2d(act1, 2, 2)

conv3 = tf.layers.conv2d(conv1, 64, 3, activation=None)
batch2 = tf.layers.batch_normalization(conv3)
act1 = tf.nn.relu(batch2) 
conv3 = tf.layers.max_pooling2d(act1, 2, 2)

fc1 = tf.contrib.layers.flatten(conv3)

fc1 = tf.layers.dense(fc1, 1024,activation=tf.nn.relu)

out = tf.layers.dense(fc1, 10,activation=None)
    
l_out = tf.nn.softmax(out)

y_predicted = tf.argmax(l_out, axis=-1)

In [7]:
weights = tf.trainable_variables()
weights

[<tf.Variable 'conv2d/kernel:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(1600, 1024) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32_ref>]

### Than you could simply
* define loss function manually
* compute error gradient over all weights
* define updates
* But that's a whole lot of work and life's short
  * not to mention life's too short to wait for SGD to converge

Instead, we shall use Tensorflow builtins

In [8]:
# Mean categorical crossentropy as a loss function
# - similar to logistic loss but for multiclass targets
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=target_y, logits=out, name="softmax_loss"))

In [9]:
accuracy, update_accuracy = tf.metrics.accuracy(target_y, y_predicted)
tf.local_variables()

[<tf.Variable 'accuracy/total:0' shape=() dtype=float32_ref>,
 <tf.Variable 'accuracy/count:0' shape=() dtype=float32_ref>]

In [10]:
optimzer = tf.train.AdamOptimizer(learning_rate=0.005)
train_step = optimzer.minimize(loss)

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [11]:
# An auxilary function that returns mini-batches for neural network training

#Parameters
# inputs - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# outputs - a vector of answers for corresponding images e.g. Y_train
#batch_size - a single number - the intended size of each batches

def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

# Training loop

Model saver.
<br>
See more:
http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/

In [12]:
model_path = "./checkpoints/model.ckpt"
saver = tf.train.Saver(max_to_keep=3)

In [14]:
import time

num_epochs = 50 # amount of passes through the data

batch_size = 500 # number of samples processed at each function call

with tf.Session() as sess:
    # initialize global wariables
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()

        sess.run(tf.local_variables_initializer())
        for batch in iterate_minibatches(X_train, y_train,batch_size):
            inputs, targets = batch

            _, train_err_batch, _ = sess.run(
                [train_step, loss, update_accuracy], 
                feed_dict={input_X: inputs, target_y:targets}
            )
            train_err += train_err_batch
            train_batches += 1
        train_acc = sess.run(accuracy)

        # And a full pass over the validation data:
        sess.run(tf.local_variables_initializer())
        for batch in iterate_minibatches(X_val, y_val, batch_size):
            inputs, targets = batch
            sess.run(update_accuracy, feed_dict={input_X: inputs, 
                                                 target_y:targets})
        val_acc = sess.run(accuracy)


        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))

        print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
        print("  train accuracy:\t\t{:.2f} %".format(
            train_acc * 100))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc * 100))
        
        # save model
        save_path = saver.save(sess, model_path, global_step=epoch)
        print("  Model saved in file: %s" % save_path)

Epoch 1 of 50 took 98.251s
  training loss (in-iteration):		0.228257
  train accuracy:		92.68 %
  validation accuracy:		98.13 %
  Model saved in file: ./checkpoints/model.ckpt-0
Epoch 2 of 50 took 93.651s
  training loss (in-iteration):		0.042535
  train accuracy:		98.68 %
  validation accuracy:		98.71 %
  Model saved in file: ./checkpoints/model.ckpt-1
Epoch 3 of 50 took 98.135s
  training loss (in-iteration):		0.029335
  train accuracy:		99.06 %
  validation accuracy:		98.79 %
  Model saved in file: ./checkpoints/model.ckpt-2
Epoch 4 of 50 took 102.362s
  training loss (in-iteration):		0.023118
  train accuracy:		99.26 %
  validation accuracy:		99.15 %
  Model saved in file: ./checkpoints/model.ckpt-3
Epoch 5 of 50 took 98.847s
  training loss (in-iteration):		0.014261
  train accuracy:		99.55 %
  validation accuracy:		98.97 %
  Model saved in file: ./checkpoints/model.ckpt-4
Epoch 6 of 50 took 102.934s
  training loss (in-iteration):		0.012703
  train accuracy:		99.58 %
  validation

Epoch 47 of 50 took 81.363s
  training loss (in-iteration):		0.007834
  train accuracy:		99.80 %
  validation accuracy:		99.05 %
  Model saved in file: ./checkpoints/model.ckpt-46
Epoch 48 of 50 took 82.015s
  training loss (in-iteration):		0.003768
  train accuracy:		99.88 %
  validation accuracy:		99.16 %
  Model saved in file: ./checkpoints/model.ckpt-47
Epoch 49 of 50 took 81.599s
  training loss (in-iteration):		0.005332
  train accuracy:		99.88 %
  validation accuracy:		99.01 %
  Model saved in file: ./checkpoints/model.ckpt-48
Epoch 50 of 50 took 81.062s
  training loss (in-iteration):		0.002057
  train accuracy:		99.95 %
  validation accuracy:		99.24 %
  Model saved in file: ./checkpoints/model.ckpt-49


Now we can restore saved parameters:

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    load_path = saver.restore(sess, saver.last_checkpoints[-1])
    print("Model restored from file: %s" % save_path)
    
    sess.run(tf.local_variables_initializer())
    for batch in iterate_minibatches(X_test, y_test, 500):
        inputs, targets = batch
        sess.run(update_accuracy, feed_dict={input_X: inputs, 
                                                   target_y:targets})
    test_acc = sess.run(accuracy)
    print("Final results:")
    print("  test accuracy:\t\t{:.2f} %".format(
        test_acc* 100))

    if test_acc * 100 > 99.5:
        print ("Achievement unlocked: 80lvl Warlock!")
    else:
        print ("We need more magic!")

INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt-49
Model restored from file: ./checkpoints/model.ckpt-49
Final results:
  test accuracy:		99.21 %
We need more magic!


# Now improve it!

* Moar layers!
* Moar units!
* Different nonlinearities!